# Chapter 9. Tests of Significance

Solutions for all exercises listed in Chapter 9 (Tests of Significance) from *Principles of Statistics* (MG Bulmer, 1965)


---

Set local environment

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

Define a function to determine if we should accept or reject the null hypothesis given a threshold.

In [2]:
def test(alpha=0.05, p_value=0):
    if p_value <= alpha:
        print('Reject null hypothesis with a significance level of', str(alpha))
    else:
        print('Unable to reject the null hypothesis with a significance level of', str(alpha))

# Exercise 9.1
In one of his experiments Mendel observed 705 plants with purple flowers and 224 plants with white flowers in plants bred from a purple-flowered $\times$ white-flowered hybrid. Test the hypothesis that the probability of a purple-flowered plant is $\frac{3}{4}$.

*Solution*

There is no reason to suppose that $p$ is greater than or less than $p=\frac{3}{4}$, so use the binomial distribution to perform a two-tailed test.

$H_0$: $p=\frac{3}{4}$

In [3]:
test(alpha=0.05, p_value=stats.binom_test(x=705, n=(705 + 224), p=0.75))

Unable to reject the null hypothesis with a significance level of 0.05


# Exercise 9.2
200 women are each given a sample of butter and a sample of margarine and asked to identify the butter; 120 of them do so correctly. Can women tell butter from margarine?

*Solution*

If women were unable to tell butter from margarine, the probability of labeling butter correctly would be 0.5. Hence, perform a two-tailed test using the binomial distribution.

$H_0$: Women cannot tell butter from margarine ($p=0.5$)

In [4]:
test(alpha=0.05, p_value=stats.binom_test(x=120, n=200, p=0.5, alternative='two-sided'))

Reject null hypothesis with a significance level of 0.05


# Exercise 9.3
In a similar test among 200 men, 108 identify the butter correctly; is there a sex difference in tase discrimination?

*Solution*

In 9.1 and 9.2, we were comparing the observed occurences against the expected number of observations. We are now comparing the difference between two proportions.

If there is no sex difference in taste discrimination, then the theoretical proportion of women who cannot tell the difference between butter and margarine ($P_w$) should be equal to that of men ($P_m$). In other words, we are testing the hypothesis that $Pw = Pm = P$.

Transforming a binomial random variable into a proportion implies dividing it by the number of observations.

$Y = \frac{X}{n} \implies E(Y) = P, V(Y) = \frac{PQ}{n}$

Hence, the statistic $p_w - p_m$ should be normally distributed with mean 0 and variance $PQ(\frac{1}{n_w} + \frac{1}{n_m})$.

We don't know the true value of $P$, so we'll estimate it with $\frac{x_w + x_m}{n_w + n_m}$

In [5]:
# Observed proportions
p_w = 120 / 200
p_m = 108 / 200

# Global proportion
p = (120 + 108) / (200 + 200)

# Standard deviation
sigma = np.sqrt(p * (1 - p) * ((1 / 200) + (1 / 200)))

# Statistic
s = (p_w - p_m) / sigma
s

1.2119357448701642

In [6]:
p_value = 2 * (1 - stats.norm(loc=0, scale=1).cdf(s))
test(alpha=0.05, p_value=p_value)

Unable to reject the null hypothesis with a significance level of 0.05


# Exercise 9.4
To test whether it is of advantage to klin-dry barley before sowing, eleven varieties of barley were sown (both klin-dried and not klin-dried). Test whether there is any advantage in klin-drying.

The yields, in lb. head corn per acre, are given below:

In [7]:
df = pd.DataFrame({'dried':[2009,1915,2011,2463,2180,1925,2122,1482,1542,1443,1535],
                   'not_dried':[1903,1935,1910,2496,2108,1961,2060,1444,1612,1316,1511]})
df

,dried,not_dried
0,2009,1903
1,1915,1935
2,2011,1910
3,2463,2496
4,2180,2108
5,1925,1961
6,2122,2060
7,1482,1444
8,1542,1612
9,1443,1316


$H_0: \mu_{dry} = \mu_{wet}$

We can use the $t$ statistic to do a two-tailed test on the difference between both populations (it is feasible that drying has a negative effect). Note that the samples were taken from the same plants, so they are *not* independent.

In [8]:
# New column with difference
df['diff'] = df['dried'] - df['not_dried']

# Use ttest_1samp since samples are not independent
t_test = stats.ttest_1samp(df['diff'], popmean=0)

# Test using p-value
test(alpha=0.05, p_value=t_test[1])

Unable to reject the null hypothesis with a significance level of 0.05


# Exercise 9.5
Use the data in Table 22 on p. 210 to test (a) whether there is an increase in comb-growth capons receiving $\frac{1}{2}$ mg androsterone, (b) whether there is an increase in capons receiving 4 mg, (c) whether there is any difference in capons receiving 4 mg and 8 mg.


In [9]:
df = pd.DataFrame({'half':[8,1,1,3,1], 'four':[17,14,14,19,13],'eight':[17,17,20,18,15]})
df

,half,four,eight
0,8,17,17
1,1,14,17
2,1,14,20
3,3,19,18
4,1,13,15


`scipy.stats.ttest_1samp` does not have one-tailed tests. Hence, we must define the corresponding $t$ distribution to compare the alpha and p-values.

(a) $H_0: \mu_0 = 0$, $H_1: \mu_0 \geq 0$

In [10]:
# Declare statistic
T = stats.ttest_1samp(df['half'], popmean=0)[0]

# Declare p-value
p = 1 - stats.t(df=len(df) - 1, loc=0, scale=1).cdf(T)

# Test H_0
test(alpha=0.05, p_value=p)

Unable to reject the null hypothesis with a significance level of 0.05


(b) $H_0: \mu_0 = 0, H_1: \mu_0 \geq 0$

In [11]:
# Declare statistic
T = stats.ttest_1samp(df['four'], popmean=0)[0]

# Declare p-value
p = 1 - stats.t(df=len(df) - 1, loc=0, scale=1).cdf(T)

# Test H_0
test(alpha=0.05, p_value=p)

Reject null hypothesis with a significance level of 0.05


(c) $H_0: \mu_{4mg} = \mu_{8mg}, H_1: \mu_{4mg} \neq \mu_{8mg}$

Since the samples are independent, we must calculate a $T$ statistic assuming that both samples come from a distribution with equal variance and perhaps different means.


In [12]:
# Declare column with differences
df['diff'] = df['four'] - df['eight']

# Declare t_test
t_test = stats.ttest_ind(a=df['four'], b=df['eight'], equal_var=True)

# Test H_0
test(alpha=0.05, p_value=t_test[1])

Unable to reject the null hypothesis with a significance level of 0.05
